In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pd.set_option('min_rows', 1000)

In [3]:
profit_df_2021 = pd.read_excel(r'C:\Users\Mykola\Desktop\rollun\profit\AmazonProfit2021JanMay.xlsx').drop_duplicates()

profit_df_april = pd.read_excel(r'C:\Users\Mykola\Desktop\rollun\new\2021_04_AmazonProfit for Analyst April.xlsx').drop_duplicates()
profit_df_april.columns = ['order id w/o doubles', 'Gross Profit']

profit_df_june = pd.read_excel(r'C:\Users\Mykola\Desktop\rollun\new\2021_06_AmazonProfit for Analytic.xlsx').drop_duplicates()
profit_df_june.columns = ['order id w/o doubles', 'Gross Profit']

frames = [profit_df_2021, profit_df_april, profit_df_june]

profit_df = pd.concat(frames)

In [4]:
Jan_Feb_df = pd.read_excel(r'C:\Users\Mykola\Desktop\rollun\Az.xlsx')
April_df = pd.read_excel(r'C:\Users\Mykola\Desktop\rollun\new\April.xlsx')
May_df = pd.read_excel(r'C:\Users\Mykola\Desktop\rollun\new\May.xlsx')
June_df = pd.read_excel(r'C:\Users\Mykola\Desktop\rollun\new\June.xlsx')

In [5]:
frames = [April_df, May_df, June_df, Jan_Feb_df]

sales_df = pd.concat(frames)

In [6]:
sales_df = sales_df[(sales_df["order-status"] != 'Cancelled') & (sales_df["fulfillment-channel"] != 'Amazon')]
sales_df = sales_df[["asin", "amazon-order-id","ship-state","sku","item-price", "purchase-date"]]

In [7]:
date1 = "2020-12-31"
date2 = "2021-07-01"

In [8]:
sales_df = sales_df[(sales_df['purchase-date'] > date1) & (sales_df['purchase-date'] < date2)]

In [9]:
z = pd.merge(sales_df, profit_df, left_on=["amazon-order-id"], right_on=['order id w/o doubles'], how='inner').drop_duplicates()

In [10]:
category_listing_df = pd.read_csv(r'C:\Users\Mykola\Desktop\rollun\Category Listing_full_table.csv')
category_listing_df = category_listing_df[["Product Type", "Product ID"]]

c:\users\mykola\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (1,2,3,4,5,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,

In [11]:
a = pd.merge(z, category_listing_df, left_on=["asin"], right_on=['Product ID'], how='left').drop_duplicates()

In [12]:
amazon_inventory_df = pd.read_csv(r'C:\Users\Mykola\Desktop\rollun\AmazonInventory_full_table.csv')

conditions = [
    (amazon_inventory_df[["ct_map_price"]] > 0),
    (amazon_inventory_df[["ct_map_price"]] < 0),
]

choices = [True, False]
amazon_inventory_df[["map_used"]] = np.select(conditions, choices, default=None)
amazon_inventory_df = amazon_inventory_df[["msin", "rid","map_used","ct_map_price"]]

In [13]:
#amazon_inventory_df = pd.read_csv(r'C:\Users\Mykola\Desktop\rollun\AmazonInventory_full_table.csv')
#o = amazon_inventory_df[["ct_map_price", "msin"]].drop_duplicates()

#o.sort_values(['ct_map_price'], ascending=[False])

#o1 = (o.loc[o['ct_map_price'].isnull()])
#o2 = (o1.loc[o['ct_map_price'] < 1])
#o1

In [14]:
b = pd.merge(a, amazon_inventory_df, left_on=["asin"], right_on=['msin'], how='left').drop_duplicates()

In [15]:
central_catalog_df = pd.read_csv(r'C:\Users\Mykola\Desktop\rollun\Central Catalog_full_table.csv')
central_catalog_df = central_catalog_df[["id", "brand_id"]]

In [16]:
c = pd.merge(b, central_catalog_df, left_on=["rid"], right_on=['id'], how='left').drop_duplicates()

In [17]:
dimension_store_df = pd.read_csv(r'C:\Users\Mykola\Desktop\rollun\Dimension Store_full_table.csv')
dimension_store_df = dimension_store_df[["id", "width", "height", "length", "weight", "is_dangerous","image"]]
dimension_store_df['is_image'] = np.where(dimension_store_df['image'].notnull(), True, False)

In [18]:
d = pd.merge(c, dimension_store_df, left_on=["rid"], right_on=['id'], how='left').drop_duplicates()

In [19]:
map_rules_df = pd.read_csv(r'C:\Users\Mykola\Desktop\rollun\Map Rules_full_table.csv')
map_rules_df = map_rules_df[["brand"]]
map_rules_df['map'] = np.where(map_rules_df['brand']!= '', True, False)
map_rules_df = map_rules_df.drop_duplicates()

In [20]:
e = pd.merge(d, map_rules_df, left_on=["brand_id"], right_on=['brand'], how='left').drop_duplicates()

In [21]:
e = e[["asin", "amazon-order-id","ship-state","sku","item-price", "rid", "Product Type","brand_id", "width", "height", "length", "weight", "is_dangerous", 'map', 'is_image', "purchase-date","Gross Profit", "map_used","ct_map_price"]]

In [22]:
p1 = pd.read_excel(r'C:\Users\Mykola\Desktop\rollun\2021-07-13_PartsUnlimited.xlsx')
p1 = p1[['mpOrderNumber']]
p1['Sr name'] = np.where(p1['mpOrderNumber']!= '', 'PartsUnlimited', True)
p1['is_dropship'] = np.where(p1['mpOrderNumber']!= '', "Pickup", True)

p2 = pd.read_excel(r'C:\Users\Mykola\Desktop\rollun\2021-07-13_RollunOffice.xlsx')
p2 = p2[['mpOrderNumber']]
p2['Sr name'] = np.where(p2['mpOrderNumber']!= '', 'RollunOffice', True)
p2['is_dropship'] = np.where(p2['mpOrderNumber']!= '', "Pickup", True)

p3 = pd.read_excel(r'C:\Users\Mykola\Desktop\rollun\2021-07-13_TuckerRocky.xlsx')
p3 = p3[['mpOrderNumber']]
p3['Sr name'] = np.where(p3['mpOrderNumber']!= '', 'TuckerRocky', True)
p3['is_dropship'] = np.where(p3['mpOrderNumber']!= '', "Pickup", True)

p4 = pd.read_excel(r'C:\Users\Mykola\Desktop\rollun\2021-07-13_RockyMountain.xlsx')
p4 = p4[['mpOrderNumber']]
p4['Sr name'] = np.where(p4['mpOrderNumber']!= '', 'RockyMountain', True)
p4['is_dropship'] = np.where(p4['mpOrderNumber']!= '', "Pickup", True)

dropship_price_info_df = pd.read_excel(r'C:\Users\Mykola\Desktop\rollun\dropship.xlsx')
dropship_price_info_df = dropship_price_info_df[["MP order number", "Sr name"]]
dropship_price_info_df['is_dropship'] = np.where(dropship_price_info_df['MP order number']!= '', "Dropship", False)
dropship_price_info_df[['mpOrderNumber']] = dropship_price_info_df[['MP order number']]
dropship_price_info_df = dropship_price_info_df[['mpOrderNumber','Sr name','is_dropship']]

frames = [p1, p2, p3, p4, dropship_price_info_df]

p = pd.concat(frames)

In [23]:
f = pd.merge(e, p, left_on=["amazon-order-id"], right_on=['mpOrderNumber'], how='left').drop_duplicates()

In [24]:
j = f[f["is_dropship"] == "Pickup"]
y = f[f["is_dropship"] != "Pickup"]

j["Gross Profit"] = j["Gross Profit"] - 1

frames = [j, y]

f = pd.concat(frames)

c:\users\mykola\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [25]:
#f = f[(f['Sr name'] == "PartsUnlimited")]

In [26]:
persent = 0.75

date1 = "2020-05-01"
date2 = "2021-07-01"


f1 = f[(f['purchase-date'] > date1) & (f['purchase-date'] <= date2) & (f['is_dropship'] == 'Pickup')] 


df5 = f1['Gross Profit'].groupby(f1['asin']).agg('sum').reset_index(name='sum')
df6 = f1['Gross Profit'].groupby(f1['asin']).agg('count').reset_index(name='count')
df = pd.merge(df5, df6, left_on=["asin"], right_on=['asin'], how='inner').drop_duplicates()
df = df.sort_values(by=['sum'], ascending=False).reset_index()
del df['index']


df1 = pd.DataFrame({'asin': [], 'sum': []})
b = df["sum"].sum()*persent
a = 0
for index, row in df.iterrows():
    
    if a < b:
        df1 = df1.append(row, ignore_index=True)
        a = a + row['sum']      
        
df1['group'] = 'A'  

df2= pd.merge(df, df1, left_on=["asin"], right_on=['asin'], how='left').drop_duplicates()
df2['group'] = df2['group'].fillna("B")
del df2['sum_y']
df2.rename(columns={'sum_x': 'sum'}, inplace=True)
del df2['count_y']
df2.rename(columns={'count_x': 'count'}, inplace=True)

o = (df2.loc[df2['group'] == "B"]).agg('count').reset_index(name='count').iloc[1:]
o.columns = ['asin', 'count']
o['asin'] = o['asin'].replace(['count'],'OTHERS')
o = (o.loc[o['asin'] == "OTHERS"])

z = (df2.loc[df2['group'] == "B"]).agg('sum').reset_index(name='sum').iloc[1:]
z.columns = ['asin', 'sum']
z['asin'] = z['asin'].replace(['sum'],'OTHERS')
z = z.loc[z['asin'] == "OTHERS"]

x = pd.merge(o, z, left_on=["asin"], right_on=['asin'], how='left').drop_duplicates()

df3 = (df2.loc[df2['group'] == "A"])
df4 = df3[["asin","sum","count"]]

frames1 = [x, df4]
p1 = pd.concat(frames1)

print("")
print("")
print("Profit by products.", p1["asin"].count(), "products generate", persent*100, "% of profit.", "Date Between ", date1, " and ", date2)


del p1['count']
del p1['sum']
p1


amazon_inventory_df1 = amazon_inventory_df[["msin", "rid"]]
p2 = pd.merge(p1, amazon_inventory_df1, left_on=["asin"], right_on=['msin'], how='inner').drop_duplicates()
del p2['asin']
del p2['msin']
p2 = p2.reset_index()
p2



Profit by products. 425 products generate 75.0 % of profit. Date Between  2020-05-01  and  2021-07-01


,index,rid
0,0,XL4KM
1,1,8A82V
2,2,FV93Z
3,3,CPYDC
4,4,FDIBM
5,5,M3557
6,6,BMDMS
7,7,7YI5A
8,8,SXFKB
9,9,5FKWK


In [27]:
montain_df = pd.read_csv(r'C:\Users\Mykola\Desktop\rollun\new\Rocky Mountain with Rollun ID_full_table.csv')

c:\users\mykola\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (4,34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
autodist_df = pd.read_csv(r'C:\Users\Mykola\Desktop\rollun\new\Autodist with Rollun ID_full_table.csv')

c:\users\mykola\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (21,72,74) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
tucker_df = pd.read_csv(r'C:\Users\Mykola\Desktop\rollun\new\Tucker Rocky with Rollun ID_full_table.csv')

c:\users\mykola\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (0,4,24,26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
parts_df = pd.read_csv(r'C:\Users\Mykola\Desktop\rollun\new\Parts.csv')
parts_df.columns = ['s_quantity_parts', 's_price_in_parts', 's_brand_id', 'rollun_id']

In [31]:
autodist_df = autodist_df[['s_quantity','s_price_in','rollun_id']]
autodist_df.columns = ['s_quantity_autodist', 's_price_in_autodist', 'rollun_id']

In [32]:
tucker_df = tucker_df[['s_quantity','s_price_in','rollun_id']]
tucker_df.columns = ['s_quantity_tucker', 's_price_in_tucker', 'rollun_id']

In [33]:
montain_df = montain_df[['s_quantity','s_price_in','rollun_id']]
montain_df.columns = ['s_quantity_montain', 's_price_in_montain', 'rollun_id']

In [34]:
p3 = pd.merge(parts_df, p2, left_on=["rollun_id"], right_on=['rid'], how='inner').drop_duplicates()
p4 = pd.merge(p3, autodist_df, left_on=["rid"], right_on=['rollun_id'], how='left').drop_duplicates()
p5 = pd.merge(p4, tucker_df, left_on=["rid"], right_on=['rollun_id'], how='left').drop_duplicates()
p6 = pd.merge(p5, montain_df, left_on=["rid"], right_on=['rollun_id'], how='left').drop_duplicates()
p6 = p6[['index','rid' ,'s_brand_id','s_price_in_parts',
        's_price_in_autodist', 
        's_price_in_tucker', 
       's_price_in_montain', 's_quantity_parts','s_quantity_autodist','s_quantity_tucker','s_quantity_montain' ]].sort_values(['index'], ascending=[True])

In [35]:
p6['s_price_in_parts_new'] = p6['s_price_in_parts']*0.95

In [42]:
print("")
print("")
print("Compare price different suppliers for top 400 products between ", date1, " and ", date2)

p6



Compare price different suppliers for top 400 products between  2020-05-01  and  2021-07-01


,index,rid,s_brand_id,s_price_in_parts,s_price_in_autodist,s_price_in_tucker,s_price_in_montain,s_quantity_parts,s_quantity_autodist,s_quantity_tucker,s_quantity_montain,s_price_in_parts_new
97,7,7YI5A,CUSTOMDYNAMICS,101.85,NaN,NaN,NaN,10,NaN,NaN,NaN,96.7575
157,12,T5CWI,UNIFILTER,19.20,17.25,19.17,15.24,10,9.0,0.0,5.0,18.2400
146,14,WYVW8,UNIFILTER,21.60,19.41,21.57,17.13,7,0.0,0.0,1.0,20.5200
118,27,4AITW,CANYONDANCER,5.70,NaN,NaN,6.87,10,NaN,NaN,22.0,5.4150
152,35,4PRYA,UNIFILTER,20.40,18.33,20.37,19.46,0,0.0,0.0,22.0,19.3800
114,40,TK2ZA,MOTIONPRO,51.20,48.50,46.62,54.08,0,9.0,0.0,22.0,48.6400
141,46,8452B,CHRISPRODUCTS,5.30,NaN,5.15,NaN,10,NaN,9.0,NaN,5.0350
67,49,CWZXH,HIFLOAIRANDOILFILTERS,10.70,9.41,10.72,11.16,0,0.0,0.0,0.0,10.1650
71,55,L2WZ4,DYNOJET,279.50,NaN,279.50,NaN,10,NaN,4.0,NaN,265.5250
163,57,FA3UN,NGK,1.89,1.88,2.32,2.63,10,0.0,0.0,0.0,1.7955


In [37]:
p6.to_excel(r'C:\Users\Mykola\Desktop\rollun\profit\PU1.xlsx', sheet_name='Compare_prices_Jan_June_21', index = False)

In [43]:
print("")
print("")
print("Compare price different suppliers for top 400 products where PU price with discount cheaper between ", date1, " and ", date2)

p6[['index','rid' ,'s_brand_id','s_price_in_parts', 's_price_in_parts_new',
        's_price_in_autodist', 
        's_price_in_tucker', 
       's_price_in_montain', 's_quantity_parts','s_quantity_autodist','s_quantity_tucker','s_quantity_montain' ]].where((p6['s_price_in_parts_new'] < p6['s_price_in_autodist'])& (p6['s_price_in_parts'].notnull())).sort_values(['s_price_in_parts'], ascending=[False])



Compare price different suppliers for top 400 products where PU price with discount cheaper between  2020-05-01  and  2021-07-01


,index,rid,s_brand_id,s_price_in_parts,s_price_in_parts_new,s_price_in_autodist,s_price_in_tucker,s_price_in_montain,s_quantity_parts,s_quantity_autodist,s_quantity_tucker,s_quantity_montain
9,389.0,HY9NR,MICHELIN,125.75,119.4625,120.81,NaN,119.04,10.0,0.0,NaN,6.0
11,148.0,LX5CI,MICHELIN,114.79,109.0505,112.79,NaN,101.06,0.0,0.0,NaN,0.0
5,167.0,AIP39,MICHELIN,105.08,99.8260,100.77,NaN,100.73,0.0,0.0,NaN,0.0
100,176.0,89BC7,NGK,8.91,8.4645,9.20,11.33,11.08,10.0,5.0,9.0,11.0
99,215.0,R31F3,NGK,8.91,8.4645,9.20,11.32,10.94,10.0,18.0,9.0,7.0
163,57.0,FA3UN,NGK,1.89,1.7955,1.88,2.32,2.63,10.0,0.0,0.0,0.0
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
d = p6[(p6['s_price_in_autodist'].notnull()) & (p6['s_quantity_parts']>0)]
df78 = d['s_brand_id'].groupby(d['s_brand_id']).agg('count').reset_index(name='count').sort_values(['count'], ascending=[False])

print("")
print("")
print("Count products per brand from PU for top 400 products between ", date1, " and ", date2)

df78



Count products per brand from PU for top 400 products between  2020-05-01  and  2021-07-01


,s_brand_id,count
10,UNIFILTER,13
2,HIFLOAIRANDOILFILTERS,10
7,MOTIONPRO,8
8,NGK,3
6,MICHELIN,2
9,TWINAIR,2
0,EPI,1
1,GATES,1
3,ITPTIRES,1
4,JTSPROCKET,1


In [40]:
df78.to_excel(r'C:\Users\Mykola\Desktop\rollun\profit\PU5.xlsx', sheet_name='Count_top_products_along_brands', index = False)